<a href="https://colab.research.google.com/github/hamza-bangash/DeepLearningPractice/blob/main/ANN/age_gender_predicaitons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AGE and Gender Predications**

## Downloading DAta From Kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
100% 330M/331M [00:00<00:00, 693MB/s]
100% 331M/331M [00:00<00:00, 722MB/s]


In [4]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

## Imp Library

In [7]:
import os
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Converting Data into DataFrame

In [8]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [9]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [10]:
len(age)

23708

In [11]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [12]:
df.shape

(23708, 3)

In [13]:
df.head()

,age,gender,img
0,28,0,28_0_3_20170119194509114.jpg.chip.jpg
1,37,0,37_0_2_20170116192009515.jpg.chip.jpg
2,7,0,7_0_0_20170110215335739.jpg.chip.jpg
3,37,0,37_0_3_20170120221511381.jpg.chip.jpg
4,43,0,43_0_3_20170119162826244.jpg.chip.jpg


## Spliting Data

In [14]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [15]:
train_df.shape

(20000, 3)

In [16]:
test_df.shape

(3708, 3)

## Data Augmatation

In [17]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


## Import ResNet 50 without top layer

In [19]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

## Connect ResNet With Multiout Model

In [21]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [22]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [23]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [ ]:
def custom_generator(generator):
    for x, y in generator:
        yield x, (y[0], y[1])

history = model.fit(custom_generator(train_generator),
                    steps_per_epoch=len(train_generator),
                    epochs=10,
                    validation_data=custom_generator(test_generator),
                    validation_steps=len(test_generator))

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 234s 350ms/step - age_loss: 16.3600 - age_mae: 16.3600 - gender_accuracy: 0.5049 - gender_loss: 1.6317 - loss: 177.9018 - val_age_loss: 14.8871 - val_age_mae: 14.8840 - val_gender_accuracy: 0.5324 - val_gender_loss: 0.6913 - val_loss: 83.3220
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 256s 411ms/step - age_loss: 15.0460 - age_mae: 15.0460 - gender_accuracy: 0.5162 - gender_loss: 0.6951 - loss: 83.8585 - val_age_loss: 14.6156 - val_age_mae: 14.6171 - val_gender_accuracy: 0.5324 - val_gender_loss: 0.6913 - val_loss: 83.0575
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 262s 419ms/step - age_loss: 14.7620 - age_mae: 14.7620 - gender_accuracy: 0.5178 - gender_loss: 0.6944 - loss: 83.5061 - val_age_loss: 14.5467 - val_age_mae: 14.5439 - val_gender_accuracy: 0.5348 - val_gender_loss: 0.6907 - val_loss: 82.9277
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 262s 420ms/step - age_loss: 14.9378 - age_mae: 14.9378 - gender_accuracy: 0.5224 - gender_loss: 0.6962 - loss: 83